---
title: Assignment 03
author:
  - name: Taylor Luckenbill
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: '2024-11-21'
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
date-modified: today
date-format: long
execute:
  echo: false
  eval: false
  freeze: auto
---

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from pyspark.sql import SparkSession
import re
import numpy as np
import plotly.graph_objects as go
from pyspark.sql.functions import col, split, explode, regexp_replace, transform, when
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id

In [2]:
np.random.seed(2)

pio.renderers.default = "notebook"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("data/lightcast_job_postings.csv")
df.createOrReplaceTempView("job_postings")

# Show Schema and Sample Data
#print("---This is Diagnostic check, No need to print it in the final doc---")

#df.printSchema() # comment this line when rendering the submission
df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 01:09:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/24 01:10:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

In [11]:
df = (
    df.withColumn("SALARY_FROM", col("SALARY_FROM").cast("float"))
      .withColumn("SALARY_TO", col("SALARY_TO").cast("float"))
      .withColumn("SALARY", col("SALARY").cast("float"))
      .withColumn("MIN_YEARS_EXPERIENCE", col("MIN_YEARS_EXPERIENCE").cast("float"))
      .withColumn("MAX_YEARS_EXPERIENCE", col("MAX_YEARS_EXPERIENCE").cast("float"))
)


In [8]:
# define a method
def compute_median(sdf,col_name):
    q = sdf.approxQuantile(col_name,[0.5],0.01)
    return q[0] if q else None

In [12]:
median_from = compute_median(df,"SALARY_FROM")
median_to = compute_median(df, "SALARY_TO")
median_salary = compute_median(df, "SALARY")

print("medians: ", median_from, median_to, median_salary)

medians:  87295.0 130042.0 115024.0


In [14]:
df = df.fillna({
    "SALARY_FROM": median_from,
    "SALARY_TO": median_to,
    "SALARY": median_salary})


In [15]:
df = df.withColumn("Average Salary",(col("SALARY_FROM")+col("SALARY_TO"))/2)

In [19]:
print(df.columns) 

['ID', 'LAST_UPDATED_DATE', 'LAST_UPDATED_TIMESTAMP', 'DUPLICATES', 'POSTED', 'EXPIRED', 'DURATION', 'SOURCE_TYPES', 'SOURCES', 'URL', 'ACTIVE_URLS', 'ACTIVE_SOURCES_INFO', 'TITLE_RAW', 'BODY', 'MODELED_EXPIRED', 'MODELED_DURATION', 'COMPANY', 'COMPANY_NAME', 'COMPANY_RAW', 'COMPANY_IS_STAFFING', 'EDUCATION_LEVELS', 'EDUCATION_LEVELS_NAME', 'MIN_EDULEVELS', 'MIN_EDULEVELS_NAME', 'MAX_EDULEVELS', 'MAX_EDULEVELS_NAME', 'EMPLOYMENT_TYPE', 'EMPLOYMENT_TYPE_NAME', 'MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'IS_INTERNSHIP', 'SALARY', 'REMOTE_TYPE', 'REMOTE_TYPE_NAME', 'ORIGINAL_PAY_PERIOD', 'SALARY_TO', 'SALARY_FROM', 'LOCATION', 'CITY', 'CITY_NAME', 'COUNTY', 'COUNTY_NAME', 'MSA', 'MSA_NAME', 'STATE', 'STATE_NAME', 'COUNTY_OUTGOING', 'COUNTY_NAME_OUTGOING', 'COUNTY_INCOMING', 'COUNTY_NAME_INCOMING', 'MSA_OUTGOING', 'MSA_NAME_OUTGOING', 'MSA_INCOMING', 'MSA_NAME_INCOMING', 'NAICS2', 'NAICS2_NAME', 'NAICS3', 'NAICS3_NAME', 'NAICS4', 'NAICS4_NAME', 'NAICS5', 'NAICS5_NAME', 'NAICS6', 'NAIC

In [17]:
#parse
export_cols = [
    "EDUCATION_LEVELS_NAME",
    "REMOTE_TYPE_NAME",
    "MAX_YEARS_EXPERIENCE",
    "Average_ Salary",
    "LOT_V6_SPECIALIZED_OCCUPATION_NAME"
]




In [18]:
df_selected = df.select(*export_cols)

{"ts": "2025-09-24 02:18:36.702", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Average_ Salary` cannot be resolved. Did you mean one of the following? [`Average Salary`, `ACTIVE_URLS`, `COMMON_SKILLS`, `CIP2_NAME`, `CIP4_NAME`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor53.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o242.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Average_ Salary` cannot be resolved. Did you mean one of the following? [`Average Salary`, `ACTIVE_URLS`, `COMMON_SKILLS`, `CIP2_NAME`, `CIP4_NAME`]. SQLSTATE: 42703;\n'Project [EDUCATION_LEVELS_NAME#21, REMOTE_TYPE_NAME#33, MAX_YEARS_EXPERI

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Average_ Salary` cannot be resolved. Did you mean one of the following? [`Average Salary`, `ACTIVE_URLS`, `COMMON_SKILLS`, `CIP2_NAME`, `CIP4_NAME`]. SQLSTATE: 42703;
'Project [EDUCATION_LEVELS_NAME#21, REMOTE_TYPE_NAME#33, MAX_YEARS_EXPERIENCE#698, 'Average_ Salary, LOT_V6_SPECIALIZED_OCCUPATION_NAME#104]
+- Project [ID#0, LAST_UPDATED_DATE#1, LAST_UPDATED_TIMESTAMP#2, DUPLICATES#3, POSTED#4, EXPIRED#5, DURATION#6, SOURCE_TYPES#7, SOURCES#8, URL#9, ACTIVE_URLS#10, ACTIVE_SOURCES_INFO#11, TITLE_RAW#12, BODY#13, MODELED_EXPIRED#14, MODELED_DURATION#15, COMPANY#16, COMPANY_NAME#17, COMPANY_RAW#18, COMPANY_IS_STAFFING#19, EDUCATION_LEVELS#20, EDUCATION_LEVELS_NAME#21, MIN_EDULEVELS#22, MIN_EDULEVELS_NAME#23, MAX_EDULEVELS#24, ... 107 more fields]
   +- Project [ID#0, LAST_UPDATED_DATE#1, LAST_UPDATED_TIMESTAMP#2, DUPLICATES#3, POSTED#4, EXPIRED#5, DURATION#6, SOURCE_TYPES#7, SOURCES